# Preprocessamento:

+ Criar variável que identifique o anotador

+ Mover tags que começam com token ' ' (vazio)

+ Remover linhas com '\n' seguidos

+ REGEX:
    + Garantir letra e números onde tamanho for maior que 1.
    + Passar múltiplos símbolos para outra linha. Exemplo:  §3º -->  § \n 3 \n º
    + Remover pontuação de centenas dos números. Exemplo: 12.200 --> 12200

+ Visualização das sentenças com displacy (from spacy import displacy) 

+ Incluir POS tagging.

+ Tranformar o dado $x_i$ em um $x'_i$ que incorpora os 2 últimos e próximos tokens.

# Classificador

+ Visualização: Separar o conjunto de test em 2 ou 3 arquivos e visualizar o que o modelo classificou e o que os anotadores classificaram (separar por id do anotador).

+ Parâmetros utilizados no classificador.

+ Analisar o formato dos dados que tem maior acerto e menor acerto tambem.

+ Para criar um contexto no erro imprimir 10 palavras antes e depois de dois erros.


# Instruções para os anotadores:

+ Atentar à marcação de tags que envolve espaço 

+ Atentar para não incluir espaço no início da Tag

# Organização do diretório: 

+ Manter toda a análise em somente um diretório

+ Formato do diretório com os datasets: /resources/dataset/

+ Notebooks:
    + '01 - Processamento.ipynb'
        + Gerar 'treino.csv' e teste.csv' para processamento
    + '02 - [CRF].ipynb' - Criando o modelo
        + Usar arquivo dos dados preprocessados gerado pelo notebook 1.
        + Gerar modelo (xxx.model)
    + '03 - Metricas.ipynb'

In [17]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
from collections import Counter

import scipy.stats
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.feature_extraction.text import HashingVectorizer

In [18]:
# Encontra todos os csv's dentro das pastas de 'mock'

extension = 'csv'
all_filenames = [i for i in glob.glob('NER/*/**/***/****/*****.{}'.format(extension))]

# NER/NER_EXPORT/161704902/[PRATICA_ETAPA_1]/Documentos

In [19]:
all_filenames[:5]

['NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20141030_RE_541090_271775268.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180411_HC_138057_314087281.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20170921_HC_147683_312770301.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180503_Pet_7074_314257052.ner.csv',
 'NER/NER_EXPORT/181300055/[PRATICA_ETAPA_1]/Documentos/20180509_HC_135415_314294988.ner.csv']

In [20]:
len(all_filenames)

2239

# Preprocessamento 

In [21]:
# Cria uma tag de inicio e fim de arquivo em cada 'csv' antes de apendar todos eles.

frames = []
for all_files in all_filenames:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    df['Tag'].iloc[0] , df['Tag'].iloc[-1] = ['INICIO_ARQ', 'FIM_ARQ'] # Altera a primeira e ultima Tag desse csv
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
combined_csv = pd.concat(frames).reset_index(drop=True)
combined_csv.to_csv("combined_csv.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
combined_csv['Token'] = combined_csv['Token'].astype('str')

In [22]:
combined_csv.head(15), combined_csv.tail(20)

(             Token         Tag
 0           EMENTA  INICIO_ARQ
 1                :           O
 2                            O
 3   CONSTITUCIONAL           O
 4                .           O
 5                            O
 6       TRIBUTÁRIO           O
 7                .           O
 8                            O
 9          IMPOSTO           O
 10                           O
 11              DE           O
 12                           O
 13           RENDA           O
 14               .           O,
                                  Token       Tag
 15446379                                       O
 15446380                      dezembro         O
 15446381                                       O
 15446382                            de         O
 15446383                                       O
 15446384                          2015         O
 15446385                             .         O
 15446386                                       O
 15446387                            

In [23]:
print("Número de linhas dos arquivos concatenados:", len(combined_csv['Tag']))

Número de linhas dos arquivos concatenados: 15446399


In [24]:
combined_csv[-10:] # Conferindo se o index foi resetado

,Token,Tag
15446389,Ministro,B_Pessoa
15446390,,I_Pessoa
15446391,TEORI,I_Pessoa
15446392,,I_Pessoa
15446393,ZAVASCKI,I_Pessoa
15446394,\n,I_Pessoa
15446395,id,O
15446396,:,O
15446397,,O
15446398,20160201_HC_132184_308418258,FIM_ARQ


# Encontra parágrafo duplo no arquivo. Uma opção de separar por sentenças.

In [25]:
a_df = combined_csv #Simplifica o nome do arquivo para a função nao ficar grande demais.
starts = a_df[a_df['Token']=='\n'].index & a_df[a_df['Token'].shift(-1)=='\n'].index #Identifica os paragrafos duplos
print(u'Padrões(sentenças) encontrados:', len(starts))

Padrões(sentenças) encontrados: 159578


In [30]:
combined_csv['Sentence #'].unique()

array(['Sentence 1', 'Sentence 2', 'Sentence 3', 'Sentence 4',
       'Sentence 5', 'Sentence 6', 'Sentence 7', 'Sentence 8',
       'Sentence 9', 'Sentence 10', 'Sentence 11', 'Sentence 12',
       'Sentence 13', 'Sentence 14', 'Sentence 15', 'Sentence 16',
       'Sentence 17', 'Sentence 18', 'Sentence', 'Sentence 159579'],
      dtype=object)

In [ ]:
%%time

combined_csv['Sentence #'] = 'Sentence'

combined_csv['Sentence #'][:starts[0]+2] = 'Sentence %d'%(1) # Primeira sentença
combined_csv['Sentence #'][starts[-1]+2:] = 'Sentence %d'%(len(starts)+1) # Última sentença

for i in range(1,len(starts)):
    combined_csv['Sentence #'][starts[i-1]+2:starts[i]+2] = 'Sentence %d'%(i+1) 

combined_csv.head(), combined_csv.tail()

In [ ]:
# Número de sentenças
len(combined_csv['Sentence #'].unique())

# Atualiza a Tag que termina com 'Doutrinador' para 'Doutrina'

In [11]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Precedente', 'I_Precedente', 'B_Pessoa', 'I_Pessoa',
       'I_Doutrina', 'FIM_ARQ', 'B_Doutrina', 'B_Doutrinador',
       'I_Doutrinador'], dtype=object)

In [12]:
# Strip o final 'dor' de todo o DataFrame (formato extremo)
# combined_csv.Tag = combined_csv.Tag.str.rstrip('dor')

In [13]:
# indices de onde a tag ocorre
indx = combined_csv[combined_csv.Tag.str.endswith('Doutrinador')].index.values

### Rodar apenas uma vez

In [14]:
%%time
# Demorado e custoso
for i in range(len(indx)):
    combined_csv.Tag.iloc[indx[i]] = combined_csv.Tag.iloc[indx[i]].rstrip('dor')

KeyboardInterrupt: 

In [15]:
combined_csv.Tag.unique()

array(['INICIO_ARQ', 'O', 'B_Ref. Legislativa', 'I_Ref. Legislativa',
       'B_Precedente', 'I_Precedente', 'B_Pessoa', 'I_Pessoa',
       'I_Doutrina', 'FIM_ARQ', 'B_Doutrina', 'I_Doutrinador',
       'B_Doutrinador'], dtype=object)

# Remove enter duplo depois de criar as sentenças

In [16]:
# Teste para ver os casoso onde ocorre enter duplo.
for i in range(len(starts)):
    print(combined_csv.iloc[starts[i]:starts[i]+2][:5])

   Token Tag  Sentence #
55    \n   O  Sentence 1
56    \n   O  Sentence 1
    Token Tag  Sentence #
322    \n   O  Sentence 2
323    \n   O  Sentence 2
    Token Tag  Sentence #
487    \n   O  Sentence 3
488    \n   O  Sentence 3
    Token Tag  Sentence #
503    \n   O  Sentence 4
504    \n   O  Sentence 4
    Token Tag  Sentence #
803    \n   O  Sentence 5
804    \n   O  Sentence 5
    Token Tag  Sentence #
819    \n   O  Sentence 6
820    \n   O  Sentence 6
    Token Tag  Sentence #
827    \n   O  Sentence 7
828    \n   O  Sentence 7
    Token Tag  Sentence #
835    \n   O  Sentence 8
836    \n   O  Sentence 8
    Token Tag  Sentence #
847    \n   O  Sentence 9
848    \n   O  Sentence 9
    Token Tag   Sentence #
855    \n   O  Sentence 10
856    \n   O  Sentence 10
    Token Tag   Sentence #
904    \n   O  Sentence 11
905    \n   O  Sentence 11
    Token Tag   Sentence #
925    \n   O  Sentence 12
926    \n   O  Sentence 12
    Token Tag   Sentence #
929    \n   O  Sentence 13
930 

      Token Tag    Sentence #
12293    \n   O  Sentence 165
12294    \n   O  Sentence 165
      Token Tag    Sentence #
12311    \n   O  Sentence 166
12312    \n   O  Sentence 166
      Token Tag    Sentence #
12321    \n   O  Sentence 167
12322    \n   O  Sentence 167
      Token Tag    Sentence #
12395    \n   O  Sentence 168
12396    \n   O  Sentence 168
      Token Tag    Sentence #
12416    \n   O  Sentence 169
12417    \n   O  Sentence 169
      Token Tag    Sentence #
12471    \n   O  Sentence 170
12472    \n   O  Sentence 170
      Token Tag    Sentence #
12653    \n   O  Sentence 171
12654    \n   O  Sentence 171
      Token Tag    Sentence #
12863    \n   O  Sentence 172
12864    \n   O  Sentence 172
      Token Tag    Sentence #
12916    \n   O  Sentence 173
12917    \n   O  Sentence 173
      Token Tag    Sentence #
12929    \n   O  Sentence 174
12930    \n   O  Sentence 174
      Token Tag    Sentence #
13111    \n   O  Sentence 175
13112    \n   O  Sentence 175
      Toke

      Token Tag    Sentence #
28745    \n   O  Sentence 322
28746    \n   O  Sentence 322
      Token Tag    Sentence #
28751    \n   O  Sentence 323
28752    \n   O  Sentence 323
      Token Tag    Sentence #
28767    \n   O  Sentence 324
28768    \n   O  Sentence 324
      Token Tag    Sentence #
28777    \n   O  Sentence 325
28778    \n   O  Sentence 325
      Token Tag    Sentence #
28966    \n   O  Sentence 326
28967    \n   O  Sentence 326
      Token Tag    Sentence #
29036    \n   O  Sentence 327
29037    \n   O  Sentence 327
      Token Tag    Sentence #
29292    \n   O  Sentence 328
29293    \n   O  Sentence 328
      Token Tag    Sentence #
29443    \n   O  Sentence 329
29444    \n   O  Sentence 329
      Token Tag    Sentence #
29621    \n   O  Sentence 330
29622    \n   O  Sentence 330
      Token Tag    Sentence #
30042    \n   O  Sentence 331
30043    \n   O  Sentence 331
      Token Tag    Sentence #
30271    \n   O  Sentence 332
30272    \n   O  Sentence 332
      Toke

      Token Tag    Sentence #
45867    \n   O  Sentence 469
45868    \n   O  Sentence 469
      Token Tag    Sentence #
46016    \n   O  Sentence 470
46017    \n   O  Sentence 470
      Token Tag    Sentence #
46135    \n   O  Sentence 471
46136    \n   O  Sentence 471
      Token Tag    Sentence #
46291    \n   O  Sentence 472
46292    \n   O  Sentence 472
      Token Tag    Sentence #
46335    \n   O  Sentence 473
46336    \n   O  Sentence 473
      Token Tag    Sentence #
46468    \n   O  Sentence 474
46469    \n   O  Sentence 474
      Token Tag    Sentence #
46513    \n   O  Sentence 475
46514    \n   O  Sentence 475
      Token Tag    Sentence #
46695    \n   O  Sentence 476
46696    \n   O  Sentence 476
      Token Tag    Sentence #
46831    \n   O  Sentence 477
46832    \n   O  Sentence 477
      Token Tag    Sentence #
47002    \n   O  Sentence 478
47003    \n   O  Sentence 478
      Token Tag    Sentence #
47181    \n   O  Sentence 479
47182    \n   O  Sentence 479
      Toke

      Token Tag    Sentence #
62155    \n   O  Sentence 622
62156    \n   O  Sentence 622
      Token Tag    Sentence #
62176    \n   O  Sentence 623
62177    \n   O  Sentence 623
      Token Tag    Sentence #
62274    \n   O  Sentence 624
62275    \n   O  Sentence 624
      Token Tag    Sentence #
62278    \n   O  Sentence 625
62279    \n   O  Sentence 625
      Token Tag    Sentence #
62381    \n   O  Sentence 626
62382    \n   O  Sentence 626
      Token Tag    Sentence #
62408    \n   O  Sentence 627
62409    \n   O  Sentence 627
      Token Tag    Sentence #
62412    \n   O  Sentence 628
62413    \n   O  Sentence 628
      Token Tag    Sentence #
62416    \n   O  Sentence 629
62417    \n   O  Sentence 629
      Token Tag    Sentence #
62438    \n   O  Sentence 630
62439    \n   O  Sentence 630
      Token Tag    Sentence #
62570    \n   O  Sentence 631
62571    \n   O  Sentence 631
      Token Tag    Sentence #
62770    \n   O  Sentence 632
62771    \n   O  Sentence 632
      Toke

...

       Token Tag     Sentence #
413108    \n   O  Sentence 4296
413109    \n   O  Sentence 4296
       Token Tag     Sentence #
413160    \n   O  Sentence 4297
413161    \n   O  Sentence 4297
       Token Tag     Sentence #
413259    \n   O  Sentence 4298
413260    \n   O  Sentence 4298
       Token Tag     Sentence #
413343    \n   O  Sentence 4299
413344    \n   O  Sentence 4299
       Token Tag     Sentence #
413393    \n   O  Sentence 4300
413394    \n   O  Sentence 4300
       Token Tag     Sentence #
413541    \n   O  Sentence 4301
413542    \n   O  Sentence 4301
       Token Tag     Sentence #
413594    \n   O  Sentence 4302
413595    \n   O  Sentence 4302
       Token Tag     Sentence #
413635    \n   O  Sentence 4303
413636    \n   O  Sentence 4303
       Token Tag     Sentence #
413765    \n   O  Sentence 4304
413766    \n   O  Sentence 4304
       Token Tag     Sentence #
414054    \n   O  Sentence 4305
414055    \n   O  Sentence 4305
       Token Tag     Sentence #
414155  

430372    \n   O  Sentence 4449
       Token Tag     Sentence #
430466    \n   O  Sentence 4450
430467    \n   O  Sentence 4450
       Token Tag     Sentence #
430507    \n   O  Sentence 4451
430508    \n   O  Sentence 4451
       Token Tag     Sentence #
430552    \n   O  Sentence 4452
430553    \n   O  Sentence 4452
       Token Tag     Sentence #
430635    \n   O  Sentence 4453
430636    \n   O  Sentence 4453
       Token Tag     Sentence #
430703    \n   O  Sentence 4454
430704    \n   O  Sentence 4454
       Token Tag     Sentence #
430786    \n   O  Sentence 4455
430787    \n   O  Sentence 4455
       Token Tag     Sentence #
430890    \n   O  Sentence 4456
430891    \n   O  Sentence 4456
       Token Tag     Sentence #
431008    \n   O  Sentence 4457
431009    \n   O  Sentence 4457
       Token Tag     Sentence #
431082    \n   O  Sentence 4458
431083    \n   O  Sentence 4458
       Token Tag     Sentence #
431099    \n   O  Sentence 4459
431100    \n   O  Sentence 4459
       T

446456    \n   O  Sentence 4609
       Token Tag     Sentence #
446569    \n   O  Sentence 4610
446570    \n   O  Sentence 4610
       Token Tag     Sentence #
446629    \n   O  Sentence 4611
446630    \n   O  Sentence 4611
       Token Tag     Sentence #
446861    \n   O  Sentence 4612
446862    \n   O  Sentence 4612
       Token Tag     Sentence #
446954    \n   O  Sentence 4613
446955    \n   O  Sentence 4613
       Token Tag     Sentence #
447050    \n   O  Sentence 4614
447051    \n   O  Sentence 4614
       Token Tag     Sentence #
447208    \n   O  Sentence 4615
447209    \n   O  Sentence 4615
       Token Tag     Sentence #
447456    \n   O  Sentence 4616
447457    \n   O  Sentence 4616
       Token Tag     Sentence #
447496    \n   O  Sentence 4617
447497    \n   O  Sentence 4617
       Token Tag     Sentence #
447634    \n   O  Sentence 4618
447635    \n   O  Sentence 4618
       Token Tag     Sentence #
447706    \n   O  Sentence 4619
447707    \n   O  Sentence 4619
       T

       Token Tag     Sentence #
462825    \n   O  Sentence 4770
462826    \n   O  Sentence 4770
       Token Tag     Sentence #
463117    \n   O  Sentence 4771
463118    \n   O  Sentence 4771
       Token Tag     Sentence #
463250    \n   O  Sentence 4772
463251    \n   O  Sentence 4772
       Token Tag     Sentence #
463272    \n   O  Sentence 4773
463273    \n   O  Sentence 4773
       Token Tag     Sentence #
463564    \n   O  Sentence 4774
463565    \n   O  Sentence 4774
       Token                 Tag     Sentence #
463709    \n  I_Ref. Legislativa  Sentence 4775
463710    \n  I_Ref. Legislativa  Sentence 4775
       Token Tag     Sentence #
463742    \n   O  Sentence 4776
463743    \n   O  Sentence 4776
       Token Tag     Sentence #
463912    \n   O  Sentence 4777
463913    \n   O  Sentence 4777
       Token Tag     Sentence #
464116    \n   O  Sentence 4778
464117    \n   O  Sentence 4778
       Token Tag     Sentence #
464137    \n   O  Sentence 4779
464138    \n   O  Senten

KeyboardInterrupt: 

In [ ]:
# Cria um array com as posições a serem retiradas.
pos = []
for i in range(len(starts)):
    pos.append(starts[i])
    pos.append(starts[i]+1)
pos[:5]

In [ ]:
# Remove as linhas do dataframe e reseta os índices.
combined_csv = combined_csv.drop(pos).reset_index(drop=True)

In [ ]:
# Confirma se a remoção foi bem sucedida.
combined_csv.head(15)

# Mover B_  com Token vazio para linha abaixo

In [ ]:
# 'begins' identifica as situações onde a Tag começa com 'B_' e o Token é vazio, uma situação onde 
# o anotador começou a marcação de um espaço vazio gerando a inconsistância.

begins = combined_csv[(combined_csv['Token']==' ') & 
                      (combined_csv['Tag'].str.startswith('B_'))].index.values

In [ ]:
begins[0], len (begins)

In [ ]:
%%time
# Rodar apenas uma vez
for i in range(len(begins)):
    combined_csv.Tag.iloc[begins[i]+1] = combined_csv.Tag.iloc[begins[i]] #Acertar a Tag do Token para começar
                                                                          #sem espaço
    combined_csv.Tag.iloc[begins[i]] = 'O' #Marca o espaço vazio como 'O'

In [ ]:
i, n = 1, 10
combined_csv.iloc[begins[i]-n+7:begins[i]+n]

# Tratar marcações que incluem vírgula no final da marcação

In [ ]:
df_teste = combined_csv.copy()

In [ ]:
# df_teste.iloc[2].shift(+1)

df_teste['Token'].shift(-1)

In [ ]:
df_teste.iloc[2]

In [ ]:
# Índice das posições onde ocorre o fim da marcação em uma vírgula
inx = df_teste[(df_teste.Tag.str.startswith('I')) & (df_teste.Token == ',') & (df_teste.Tag.shift(-1) == 'O')].index.values

In [ ]:
# Conferindo a ocorrência
i = 9
df_teste.iloc[inx[i]-5:inx[i]+10]

In [ ]:
df_teste = df_teste.drop(inx).reset_index(drop=True)

In [ ]:
# Conferindo se funcionou
i = 9
df_teste.iloc[inx[i]-20:inx[i]+15]

In [ ]:
combined_csv = df_teste.copy()

In [ ]:
combined_csv[(combined_csv.Tag.str.startswith('B')) & (combined_csv.Token == ' ')]

In [ ]:
list(combined_csv.Tag.unique())

# IMPORTANTE: Esse último passo é necessário para salvar todas as alterações feitas no preprocessamento.

In [ ]:
# Salvando o processamento feito nos dados
combined_csv.to_csv("preprocessados.csv",index=False,encoding='utf-8')

# <center> $\color{red}{\text{TESTES FALHOS}}$ </center>

## Tratar caracteres especiais

In [ ]:
# Criando um dataframe teste para não alterar o arquivo principal
df_teste = combined_csv.copy()

In [ ]:
# Identifica os tokens que começam com '§' e são maiores que 1 pois queremos pegar os casos onde
# '§' está associado ao numero do paragrafo.
espec_carac = list(df_teste[(df_teste.Token.str.startswith('§')) & (df_teste.Token.str.len()>1)].Token.unique())
espec_carac

In [ ]:
df_teste[df_teste.Token.str.match(espec_carac[0])]

### Teste com duas sentenças

In [ ]:
df_senten = df_teste[df_teste['Sentence #'] == 'Sentence 310'].append(df_teste[df_teste['Sentence #'] == 'Sentence 1589'])

df_senten.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(len(df_senten)):
    if any(carac in df_senten.Token.iloc[i] for carac in espec_carac) == True:
        line = df_senten.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, df_senten.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_senten.iloc[:i], line0, line1, line2, df_senten.iloc[i+1:]]).reset_index(drop=True)

In [ ]:
# Vendo se o novo dataframe (df_senten1) está alterado em relação ao dataframe anterior (df_senten).

In [ ]:
(df_senten1.tail(10))

In [ ]:
df_senten.tail(10)

### Para o conjunto de teste (com todas as sentenças)

In [ ]:
for i in range(len(df_teste)):
    if any(carac in df_teste.Token.iloc[i] for carac in espec_carac) == True:
        line = df_teste.iloc[i]
        splt = list(line.Token)
        
        if line.Tag.startswith('I_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line2 = line.copy()
                line2.Token = splt[2]

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

        if line.Tag.startswith('B_'):
            if len(line.Token) == 2:
                #Para tamanho 2 temos que transformar uma linha em duas
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, df_teste.iloc[i+1:]]).reset_index(drop=True)
            if len(line.Token) == 3:
                #Para tamanho 3 temos que transformar uma linha em três
                line0 = line.copy()
                line0.Token = splt[0]
                line1 = line.copy()
                line1.Token = splt[1]
                line1.Tag = line.Tag.replace('B_','I_')
                line2 = line.copy()
                line2.Token = splt[2]
                line2.Tag = line.Tag.replace('B_','I_')

                # Transforma em DataFrame
                line0 = pd.DataFrame(line0).transpose()
                line1 = pd.DataFrame(line1).transpose()
                line2 = pd.DataFrame(line2).transpose()

                df_senten1 = pd.concat([df_teste.iloc[:i], line0, line1, line2, df_teste.iloc[i+1:]]).reset_index(drop=True)

In [ ]:
df_senten1[(df_senten1.Token.str.startswith('§')) & (df_senten1.Token.str.len()>1)].head()

# Aparentemente, não está funcionando.

## Remove pontuação dos números

In [ ]:
# Criando um DataFrame onde todas os tokens possuem pontuação 
df_pont = combined_csv[(combined_csv.Token.str.contains("""[.]""")) & (combined_csv.Token.str.len()>1)]

In [ ]:
df_pont.shape

In [ ]:
# Tokens de tamanho maior que 1.
df_pont.head(30)

In [ ]:
# combined_csv.iloc[2722-3:2722+5]
df_pont[(df_pont.Tag.str.endswith('Precedente'))]

In [ ]:
df_pont.Tag.value_counts()

### Não foi possível chegar em um consenso sobre qual pontuação deveria ser removida e como localizá-la no df.

## Caso onde 'ADV' aparece ao final do Token

In [ ]:
# No meio do preprocessamento foi identificado tokens que terminam com 'adv' e estão colados ao nome do
# advogado referente ao caso.
# Tokens que terminan com 'adv'.

In [ ]:
adv = combined_csv[combined_csv.Token.str.endswith('ADV')]#.reset_index(drop=True)
adv.head()#, adv.shape

In [ ]:
# Desconfiança que o nome do advogado está agregado à palavra 'adv'

adv[adv.Token.str.len() >3]
adv_pos = adv[adv.Token.str.len() >3].index.values

k = 2 # Vê as linhas antes e depois do k-ésimo ocorrido .
combined_csv.iloc[adv_pos[k]-3:adv_pos[k]+3] # Olhando para as linhas anteriores e posteriores o acontecimento.
# combined_csv.iloc[adv_pos[0]].Token[-3:] #Separando a parte 'adv'

In [ ]:
print("Quantidade de vezes que esse caso ocorre em todos os arquivos:",len(adv_pos))